In [ ]:
#Install the transformer library
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 70.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 82.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.4/182.4 KB 22.1 MB/s eta 0:00:00


In [ ]:
# Disabling gradient computations:
import pandas as pd
import torch
torch.set_grad_enabled(False)

In [ ]:
# Pre-loading the BERT model:
from transformers import BertTokenizer
from transformers import BertForMaskedLM

MODEL_NAME = 'bert-base-cased'

tokenizer = BertTokenizer.from_pretrained(MODEL_NAME)
model = BertForMaskedLM.from_pretrained(MODEL_NAME)

Downloading:   0%|          | 0.00/213k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/436M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [5]:
df=pd.read_excel('sentences.xlsx')
df.head()

,Sentence,Masked_sentence,Expected,Unexpected
0,The woman who bakes cookies for the children i...,The woman who bakes cookies for the children [...,is,are
1,The doctor who examines the patients is over t...,The doctor who examines the patients [MASK] ov...,is,are
2,The teacher who always encourages her students...,The teacher who always encourages her students...,is,are
3,The man who fixes cars is suffering from the flu.,The man who fixes cars [MASK] suffering from t...,is,are
4,The chef who cooks delicious meals is feeling ...,The chef who cooks delicious meals [MASK] feel...,is,are


In [6]:
def compareProbabilities(row):
  instance = row['Masked_sentence']

  expected = row['Expected']
  unexpected = row['Unexpected'] 

  # convert words to indices
  inputs = tokenizer(instance, return_tensors='pt')

  # get model outputs
  outputs = model(**inputs)

  # determine position of the masked token
  mask_index = int((inputs['input_ids'][0] == tokenizer.mask_token_id).nonzero())

  # determine expected and unexpected indices
  expected_index = tokenizer.convert_tokens_to_ids(expected)
  unexpected_index = tokenizer.convert_tokens_to_ids(unexpected)

  # extract logits for expected and unexpected tokens
  expected_logit = outputs.logits[0,mask_index,expected_index]
  unexpected_logit = outputs.logits[0,mask_index,unexpected_index]

  # correct prediction returns 1, incorrect returns 0
  if expected_logit>unexpected_logit:
    print('CORRECT', instance, '| prediction:',expected,'|',(expected, expected_logit),(unexpected,unexpected_logit))
    return 1
  elif unexpected_logit > expected_logit:
    print('WRONG', instance, '| prediction:',unexpected,'|',(expected, expected_logit),(unexpected,unexpected_logit))
    return 0
  else:
    #no clear preference
    print('WRONG NO PREFERENCE', instance, (expected, expected_logit),(unexpected,unexpected_logit))
    return 0

In [7]:
# Iterate through the dataset using the function and compute the accuracy
correctCounter = 0
for index, row in df.iterrows():
  print(index)
  result = compareProbabilities(row)
  correctCounter+=result
print('Accuracy on dataset:', correctCounter / len(df))

0
CORRECT The woman who bakes cookies for the children [MASK] ill. | prediction: is | ('is', tensor(13.8958)) ('are', tensor(7.8608))
1
CORRECT The doctor who examines the patients [MASK] over there. | prediction: is | ('is', tensor(13.6654)) ('are', tensor(8.0650))
2
CORRECT The teacher who always encourages her students [MASK] feeling under the weather. | prediction: is | ('is', tensor(9.0810)) ('are', tensor(6.4414))
3
CORRECT The man who fixes cars [MASK] suffering from the flu. | prediction: is | ('is', tensor(16.1777)) ('are', tensor(8.2465))
4
CORRECT The chef who cooks delicious meals [MASK] feeling sick. | prediction: is | ('is', tensor(12.8488)) ('are', tensor(5.6307))
5
CORRECT The librarian who helps people find books [MASK] happy. | prediction: is | ('is', tensor(15.4512)) ('are', tensor(9.7371))
6
CORRECT The writer who pens bestselling novels [MASK] working at her desk. | prediction: is | ('is', tensor(15.1251)) ('are', tensor(7.6277))
7
CORRECT The salesperson who helps